### import

In [1]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmcv import Config

In [2]:
# 데이터 셋 등록
@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

In [3]:
# config file, checkpoints 파일 경로 셋팅
config_file = "/home/bgw2001/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py"
checkpoints_file = "/home/bgw2001/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"

In [4]:
cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [5]:
from mmdet.apis import set_random_seed

## config file setting
cfg.seed = 0
set_random_seed(0, deterministic=False)

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.dataset_type = "CapsuleDataset"

cfg.dataset_type = 'CapsuleDataset'
cfg.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"

cfg.data.train.type = 'CapsuleDataset'
cfg.data.train.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.train.ann_file = "train.json"
cfg.data.train.img_prefix = "train"

cfg.data.val.type = 'CapsuleDataset'
cfg.data.val.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.val.ann_file = "valid.json"
cfg.data.val.img_prefix = "valid"

cfg.data.test.type = 'CapsuleDataset'
cfg.data.test.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.test.ann_file = "test.json"
cfg.data.test.img_prefix = "test"

cfg.evaluation.interval = 1
cfg.optimizer.lr = 0.01

cfg.lr_config.policy = 'step'
cfg.lr_config.warmup = None
cfg.lr_config.step = [16, 22]
cfg.lr_config.gamma = 0.1


cfg.log_config.interval = 100
cfg.runner.max_epochs = 25

cfg.checkpoint_config.interval = 1

cfg.load_from = "/home/bgw2001/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"
cfg.work_dir = "/home/bgw2001/mmdetection/work_dir/"

cfg.gpu_ids = range(1)
cfg.data.samples_per_gpu = 8

albu_train_transforms = [
    dict(
        type='HorizontalFlip',
        p=0.5)
]

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.train_pipeline = train_pipeline
cfg.data.train.pipeline = train_pipeline

In [6]:
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.39s)
creating index...
index created!


In [8]:
datasets[0]


CapsuleDataset Train dataset with number of images 37573, and instance counts: 
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
| category     | count | category    | count | category     | count | category        | count | category      | count |
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+
| 0 [01_ulcer] | 23439 | 1 [02_mass] | 5795  | 2 [04_lymph] | 7156  | 3 [05_bleeding] | 11815 | -1 background | 0     |
+--------------+-------+-------------+-------+--------------+-------+-----------------+-------+---------------+-------+

In [9]:
datasets[0].CLASSES

('01_ulcer', '02_mass', '04_lymph', '05_bleeding')

In [10]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg = cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('01_ulcer', '02_mass', '04_lymph', '05_bleeding')


In [11]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-11-28 18:44:00,599 - mmdet - INFO - load checkpoint from /home/bgw2001/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2021-11-28 18:44:00,600 - mmdet - INFO - Use load_from_local loader


loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


2021-11-28 18:44:00,707 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
2021-11-28 18:44:00,710 - mmdet - INFO - Start running, host: bgw2001@bgw2001-All-Series, work_dir: /home/bgw2001/mmdetection/work_dir
2021-11-28 18:44:00,711 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VE

2021-11-28 19:19:23,816 - mmdet - INFO - Epoch [1][2100/4697]	lr: 1.000e-02, eta: 1 day, 8:23:02, time: 0.997, data_time: 0.013, memory: 8318, loss_rpn_cls: 0.0106, loss_rpn_bbox: 0.0068, loss_cls: 0.1238, acc: 95.9102, loss_bbox: 0.1225, loss: 0.2637
2021-11-28 19:21:03,611 - mmdet - INFO - Epoch [1][2200/4697]	lr: 1.000e-02, eta: 1 day, 8:20:13, time: 0.998, data_time: 0.013, memory: 8318, loss_rpn_cls: 0.0104, loss_rpn_bbox: 0.0070, loss_cls: 0.1210, acc: 96.0664, loss_bbox: 0.1201, loss: 0.2584
2021-11-28 19:22:43,434 - mmdet - INFO - Epoch [1][2300/4697]	lr: 1.000e-02, eta: 1 day, 8:17:32, time: 0.998, data_time: 0.013, memory: 8318, loss_rpn_cls: 0.0114, loss_rpn_bbox: 0.0070, loss_cls: 0.1145, acc: 96.1621, loss_bbox: 0.1187, loss: 0.2516
2021-11-28 19:24:23,484 - mmdet - INFO - Epoch [1][2400/4697]	lr: 1.000e-02, eta: 1 day, 8:15:06, time: 1.000, data_time: 0.013, memory: 8318, loss_rpn_cls: 0.0109, loss_rpn_bbox: 0.0071, loss_cls: 0.1205, acc: 95.9946, loss_bbox: 0.1187, loss:

[>>>>>>>>>>>>>>>>>>>>>>>>] 12524/12524, 16.4 task/s, elapsed: 765s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>] 12524/12524, 15.7 task/s, elapsed: 798s, ETA:     0s

2021-11-30 08:53:03,652 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.20s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.16s).
Accumulating evaluation results...


2021-11-30 08:53:10,398 - mmdet - INFO - Epoch(val) [25][12524]	bbox_mAP: 0.4350, bbox_mAP_50: 0.7730, bbox_mAP_75: 0.4380, bbox_mAP_s: 0.1590, bbox_mAP_m: 0.4190, bbox_mAP_l: 0.4220, bbox_mAP_copypaste: 0.435 0.773 0.438 0.159 0.419 0.422


DONE (t=1.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.773
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.438
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.223
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.538


In [12]:
cfg.dump("/home/bgw2001/mmdetection/work_dir/faster_rcnn_resnet50_conf.py")

## import

In [6]:
import json
import json
import base64
import numpy as np
import cv2
import pandas as pd
import os

from PIL import Image
from io import BytesIO
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from tqdm import tqdm
from glob import glob
from mmcv import Config

In [2]:
# 데이터 셋 등록
@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

config_file = "/home/bgw2001/mmdetection/work_dir/Faster_RCNN_Resnet50_v1/faster_rcnn_resnet50_conf.py"
checkpoint_file = "/home/bgw2001/mmdetection/work_dir/Faster_RCNN_Resnet50_v1/epoch_17.pth"

In [3]:
def submission(model, paths, score_thr = 0.5):
    file_names = []
    class_ids = []
    confidences = []
    
    point1_x = []
    point1_y = []
    
    point2_x = []
    point2_y = []

    point3_x = []
    point3_y = []

    point4_x = []
    point4_y = []
    
    for path in tqdm(paths):
        
        with open(path, "r") as f:
            data = json.load(f)
        
        img = Image.open(BytesIO(base64.b64decode(data['imageData'])))
        img = np.array(img, np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        results = inference_detector(model, img)
        
        index = path.rfind("/")
        file_name = path[index + 1:]
        
        for ind,result in enumerate(results):
            for i in range(len(result)):
                if result[i,4] > score_thr:
                    file_names.append(file_name)
                    xmin = result[i,0]
                    ymin = result[i,1]
                    xmax = result[i,2]
                    ymax = result[i,3]
            
                    point1_x.append(xmin)
                    point1_y.append(ymin)
            
                    point2_x.append(xmax)
                    point2_y.append(ymin)
            
                    point3_x.append(xmax)
                    point3_y.append(ymax)
            
                    point4_x.append(xmin)
                    point4_y.append(ymax)
            
                    class_ids.append(ind)
                    confidences.append(result[i,4])
    
    submission = pd.DataFrame({"file_name":file_names,
                               "class_id":class_ids,
                               "confidence":confidences,
                               "point1_x":point1_x,
                               "point1_y":point1_y,
                               "point2_x":point2_x,
                               "point2_y":point2_y,
                               "point3_x":point3_x,
                               "point3_y":point3_y,
                               "point4_x":point4_x,
                               "point4_y":point4_y})
    return submission

In [4]:
!pwd

/media/bgw2001/One Touch/project/HardCover_ObjectDetection/code/faster_rcnn


In [7]:
os.chdir("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/")
!ls -ail

total 1792
715 drwxr-xr-x  7 bgw2001 bgw2001 262144 11월 28 00:32 .
683 drwxr-xr-x 18 bgw2001 bgw2001 262144 11월 17 17:26 ..
723 drwxr-xr-x  5 bgw2001 bgw2001 262144 11월 28 18:52 code
721 drwxr-xr-x  8 bgw2001 bgw2001 262144 11월 25 11:24 data
726 drwxr-xr-x  2 bgw2001 bgw2001 262144 11월 25 11:40 .ipynb_checkpoints
724 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 28 18:52 practice_code
722 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 19 14:50 ref


In [8]:
model = init_detector(config_file, checkpoint_file)

paths = glob("./data/test/*.json")

Use load_from_local loader


In [9]:
print(len(paths))

20874


In [16]:
df = submission(model, paths, score_thr = 0.6)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 20874/20874 [29:04<00:00, 11.96it/s]


In [17]:
print("데이터 프레임 모양:",df.shape)
df.head()

데이터 프레임 모양: (25355, 11)


,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200000.json,3,0.786032,377.178467,346.110779,402.156799,346.110779,402.156799,373.104889,377.178467,373.104889
1,test_200001.json,0,0.985296,115.345901,328.685059,267.444366,328.685059,267.444366,467.771942,115.345901,467.771942
2,test_200001.json,0,0.962814,407.581421,193.116669,520.224670,193.116669,520.224670,312.753479,407.581421,312.753479
3,test_200002.json,1,0.748694,221.077362,345.081451,345.777252,345.081451,345.777252,490.763245,221.077362,490.763245
4,test_200003.json,1,0.997235,227.892868,139.104950,318.544739,139.104950,318.544739,255.933640,227.892868,255.933640


In [18]:
df['class_id'] = df['class_id'] + 1
df[df['file_name'] == "test_200001.json"]

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
1,test_200001.json,1,0.985296,115.345901,328.685059,267.444366,328.685059,267.444366,467.771942,115.345901,467.771942
2,test_200001.json,1,0.962814,407.581421,193.116669,520.224670,193.116669,520.224670,312.753479,407.581421,312.753479


In [19]:
df.to_csv("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/submission/faster_rcnn_resnet50_submission_v1_2.csv",index=False)